## Imports & Installs & Check GPU

In [1]:
# !pip install --upgrade --quiet numerapi 
# !pip install --quiet pyarrow 
# !pip install --quiet fastparquet


In [2]:
import pandas as pd

In [3]:
import tensorflow as tf

# Checks for the availability of GPUs 
if tf.config.list_physical_devices('GPU'):
    print('GPU is available.')
else:
    print('GPU is not available.')


GPU is available.


In [4]:
import numerapi
napi = numerapi.NumerAPI('GVTNYX7YW4BUPFPC52ZUM3C5IDT2CUU4', 'EKGP4IDIK6U2XEM3ZVY57PZXAVFG2TMMAAAS6EW7EB5EVVZAKTJZ5INGBWRLJCFS')
import numpy as np
import tensorflow as tf
import gc

## Training Data

In [5]:
df = pd.read_parquet("validation.parquet").iloc[140000:240000]


In [6]:
X_train = np.nan_to_num(df[df.columns[2:1588]].values)
y_train = np.nan_to_num(df[df.columns[1588]].values)

In [7]:
# # Define the original numpy array
# print(y_train.shape)
# # Define the number of classes
# num_classes = 5

# # Convert the array to integers
# integer_array = (y_train * (num_classes-1)).astype(int)

# # Convert the integer array to one-hot encoded form
# one_hot_array = tf.one_hot(integer_array, depth=num_classes)
# print(one_hot_array)

In [8]:

# # Find unique elements
# unique_elements = np.unique(X_train)

# # Convert numpy array to list
# unique_list = unique_elements.tolist()

# print(unique_list)

In [9]:
del df
gc.collect()

43

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# X_train, X_test, y_train, y_test = train_test_split(np_feature_array, np_target_array, test_size=0.2, random_state=42)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(type(X_train))
# X_test = scaler.fit_transform(_test)

<class 'numpy.ndarray'>


# Linear Regression Model

In [11]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Define the learning rate schedule function
def lr_schedule(epoch):
    initial_lr = 0.001
    drop_rate = 0.5
    epochs_drop = 20.0
    lr = initial_lr * np.power(drop_rate, np.floor(epoch/epochs_drop))
    return lr

# Instantiate the learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Define the optimizer with the initial learning rate
optimizer = Adam(learning_rate=0.001)

In [12]:
from tensorflow.keras import layers, regularizers, callbacks

num_features = X_train.shape[1]

reg_factor = 0.01

LRmodel = tf.keras.Sequential([layers.Dense(
        1,
        input_dim=num_features, 
        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01),
        activation='linear')
])

# Set up early stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value
)


# LRmodel.compile(loss='mean_squared_error', optimizer=optimizer)
LRmodel.compile(loss=tf.keras.losses.MSE, optimizer=optimizer)

# Train the model
history = LRmodel.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[early_stopping, lr_scheduler])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/100


2023-06-04 15:03:17.501687: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2500/2500 [==============================] - 15s 6ms/step - loss: 0.2027 - val_loss: 0.0917 - lr: 0.0010
Epoch 2/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0936 - val_loss: 0.0908 - lr: 0.0010
Epoch 3/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0948 - val_loss: 0.0916 - lr: 0.0010
Epoch 4/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0932 - val_loss: 0.0978 - lr: 0.0010
Epoch 5/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0940 - val_loss: 0.0861 - lr: 0.0010
Epoch 6/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0945 - val_loss: 0.1035 - lr: 0.0010
Epoch 7/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0937 - val_loss: 0.0895 - lr: 0.0010
Epoch 8/100
2500/2500 [==============================] - 15s 6ms/step - loss: 0.0944 - val_loss: 0.1022 - lr: 0.0010


In [13]:

del X_train
del y_train
gc.collect()

1583

In [14]:
# df = pd.read_parquet("train.parquet").iloc[100000:200000]

In [15]:
# X_train = np.nan_to_num(df[df.columns[2:1588]].values)
# y_train = np.nan_to_num(df[df.columns[1588]].values)

In [16]:
# del df
# gc.collect()
# X_train = scaler.fit_transform(X_train)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# print(type(X_train))
# # X_test = scaler.fit_transform(_test)
# history = LRmodel.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[early_stopping, lr_scheduler])

# val

In [17]:
from scipy import stats

def numerai_corr(preds, target):
  # rank (keeping ties) then Gaussianize predictions to standardize prediction distributions
  ranked_preds = (preds.rank(method="average").values - 0.5) / preds.count()
  gauss_ranked_preds = stats.norm.ppf(ranked_preds)
  
  # make targets centered around 0
  centered_target = target - target.mean()
  
  # raise both preds and target to the power of 1.5 to accentuate the tails
  preds_p15 = np.sign(gauss_ranked_preds) * np.abs(gauss_ranked_preds) ** 1.5
  target_p15 = np.sign(centered_target) * np.abs(centered_target) ** 1.5
  
  # finally return the Pearson correlation
  return np.corrcoef(preds_p15, target_p15)[0, 1]

In [18]:
# df_val = pd.read_parquet("validation.parquet").iloc[:70000]
df_val = pd.read_parquet("train.parquet").iloc[140000:240000]

In [19]:
X_test = scaler.fit_transform(np.nan_to_num(df_val[df_val.columns[2:1588]].values))
y_test = np.nan_to_num(df_val[df_val.columns[1588]].values)

del df_val
gc.collect()


0

In [20]:
# Assume `X_test` is your test features tensor and `y_test` is your test targets tensor

# Use the trained model to predict on the test set
y_pred = LRmodel.predict(X_test)
type(y_pred)
rounded_pred = np.round(y_pred * 4) / 4
print(rounded_pred)
print(y_pred.shape, rounded_pred.shape)
# print(y_pred[2], y_test[2])

# # Calculate the Mean Squared Error (MSE) between the predicted and actual results
# mse = tf.keras.losses.mean_squared_error(y_test, y_pred)

# # If mse is a tensor, you can convert it to a numpy array for further calculations or analysis
# # mse = mse.numpy()

# print("Test MSE: ", mse)
# # print(f"MSE shape:{mse.shape}")

3125/3125 [==============================] - 4s 1ms/step
[[0.5 ]
 [0.5 ]
 [0.5 ]
 ...
 [0.5 ]
 [0.5 ]
 [0.75]]
(100000, 1) (100000, 1)


In [32]:
y_pred = y_pred.flatten()
rounded_pred = rounded_pred.flatten()
k = 200
print(f"our predictions:     {np.round(y_pred[k:20-6+k], decimals=2)}")
print(f"rounded predictions: {np.round(rounded_pred[k:20-8+k], decimals=2)}")
print(f"actual results:      {y_test[k:20-6+k]}")

our predictions:     [0.46 0.34 0.43 0.57 0.57 0.77 0.44 0.38 0.36 0.54 0.6  0.48 0.4  0.57]
rounded predictions: [0.5  0.25 0.5  0.5  0.5  0.75 0.5  0.5  0.25 0.5  0.5  0.5 ]
actual results:      [0.   0.75 0.25 0.5  0.5  0.5  0.75 0.75 0.5  0.5  0.25 0.25 0.5  0.5 ]


In [22]:
print(numerai_corr(preds=pd.Series(y_pred), target=y_test))
print(numerai_corr(preds=pd.Series(rounded_pred), target=y_test))

0.01078185349394332
0.01025874699381771


In [23]:
total_mse = tf.reduce_mean(tf.keras.losses.MAE(y_test, y_pred))
print(total_mse)
print(tf.reduce_mean(tf.keras.losses.MAE(y_test, rounded_pred)))

tf.Tensor(0.19201629, shape=(), dtype=float32)
tf.Tensor(0.1796525, shape=(), dtype=float32)


In [24]:
result_array = np.array([numerai_corr(preds=pd.Series(np.random.uniform(0, 1, 100000)), target=y_test) for _ in range(2000)])

In [25]:
print("mean:", result_array.mean())
print("std dev:", np.std(result_array))

mean: -6.674273863670657e-05
std dev: 0.003168503888445623


In [26]:
# # Now, generate the numpy array
# result_array = np.array([numerai_corr(preds=pd.Series(np.random.uniform(0, 1, 70000)), target=y_test) for _ in range(2000)])
# print("mean:", result_array.mean())
# print("std dev:", np.std(result_array))

In [27]:
# del X_test
# del y_test
# gc.collect()

# Submission

In [28]:

# Get current round
current_round = napi.get_current_round()
print(current_round)

# # Download latest live features
# napi.download_dataset(f"v4.1/live_{current_round}.parquet")
live_data = pd.read_parquet(f"499_v4_1_live.parquet")
live_features = live_data[[f for f in live_data.columns if "feature" in f]]

# # Generate live predictions
live_predictions = LRmodel.predict(live_features).flatten()
live_predictions = np.round(live_predictions * 4) / 4
print(live_predictions)

# Find unique elements
unique_elements = np.unique(live_predictions)

# Convert numpy array to list
unique_list = unique_elements.tolist()

print(unique_list)


499
154/154 [==============================] - 0s 1ms/step
[0.5 0.5 0.5 ... 0.5 0.5 0.5]
[0.25, 0.5, 0.75]


In [29]:
# Format submission
submission = pd.Series(live_predictions, index=live_features.index).to_frame("prediction")
submission.to_csv(f"prediction_{current_round}.csv")


In [33]:
# Upload submission 
napi.upload_predictions(f"prediction_{current_round}.csv", model_id="6e2dad38-a2db-497f-8061-d296f2a762eb")

2023-06-04 15:07:22,596 INFO numerapi.base_api: uploading predictions...


'e917952a-4c03-4414-8949-8f06103853f6'